In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax
import jax.numpy as jnp
import jax.random as r
import distrax as dx
import matplotlib.pyplot as plt

2024-11-05 16:49:03.449914: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730825343.464352 1648633 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730825343.468590 1648633 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
import optax as ox

In [3]:
import flax.nnx as nnx

In [4]:
from typing import TypeVar

In [5]:
import sys
import os

# Navigate the current notebook directory to src directory
current_dir = os.getcwd()
jax_eco_dir = os.path.dirname(os.path.dirname(current_dir))
src_dir = os.path.join(jax_eco_dir, 'src')
sys.path.append(jax_eco_dir)

# Now you can import the module
import src.np_from_scratch as npfs

/scratch/ww347/projects/Reproductions/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
key0 = r.PRNGKey(0)

In [7]:
TRAINING_ITERATIONS = int(2e5)
MAX_CONTEXT_POINTS = 10
PLOT_AFTER = int(2e4)

In [15]:
curve_reader = npfs.GPCurvesReader(
    batch_size=8, max_num_context=MAX_CONTEXT_POINTS)

In [9]:
cnp_spec = npfs.CNPSpec(
    x_dim=1, y_dim=1, r_dim=64,
    encoder_hidden_output_sizes=[64, 64],
    decoder_hidden_output_sizes=[32, 32],
)

model = npfs.CNP(cnp_spec=cnp_spec, rngs=nnx.Rngs(0))

In [21]:
learning_rate = 5e-3
momentum = 0.9

optimizer = nnx.Optimizer(model, ox.adamw(learning_rate, momentum))
metrics = nnx.metrics.Average('log_prob')

def loss_fn(model: npfs.CNP, batch: npfs.CNPRegressionInstance):
    return -model.log_prob(batch)

@nnx.jit
def train_step(
    model: npfs.CNP, 
    optimizer: nnx.Optimizer,
    metrics: nnx.metrics.Metric, 
    batch: npfs.CNPRegressionInstance,
):
    grad_fn = nnx.value_and_grad(loss_fn)
    log_prob, grad = grad_fn(model, batch)
    # in place updates
    metrics.update(log_prob)
    optimizer.update(grad) 

@nnx.jit
def eval_step(model: npfs.CNP, metrics: nnx.metrics.Metric, batch: npfs.CNPRegressionInstance):
    log_prob = model.log_prob(batch)
    metrics.update(log_prob)

In [22]:
# test plot function out for initialised model
batch = curve_reader.generate_regression_instance(key=key0, eval_mode=True)

x_values.shape: (8, 406, 1)
y_values.shape: (8, 406, 1)


In [25]:
type(batch)

src.np_from_scratch.CNPRegressionInstance

In [34]:
preds.loc

Array([[[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       ...,

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]]], dtype=float32)

In [27]:
preds = model.predictive(batch.query)

(8, 64) (8, 400, 1)


Rather unhelpfully, the mean and standard deviation come to zero!

In [37]:
# can check that sensible initialisation has indeed occurred
# #model.encoder.layers[0].kernel

In [32]:
preds.stddev()

Array([[[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       ...,

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]]], dtype=float32)

Status:
- want to figure out why this gives nans everywhere
- reasonable to go through with python debugger and see what pops up

In [30]:
preds.event_shape

(1,)